# Ec. de calor en coordenadas polares

Vamos a simular el comportamiento de la ecuación del calor usando el método de elementos finitos tomando un disco de radio r y partiendolo en triangulos pequeños, consideraremos que la ecuacion de calor tiende a ser tipo parabolico; $\dfrac {du}{dt}-\nabla \left( c\nabla u\right) +au=f$ , pero si consideramos la ecuación $$u_{t}=k\Delta u=u_{xx}+u_{yy}$$

En un disco circular D de centro 0 y radio, por ejemplo,  1, con $k = 0.1$ con datos de frontera $u\left( x,y,t\right) =0$ dado que $\left( x,y\right) \in \partial D$ y $t > 0$, y con distribución inicial de temperatura:

$$u\left( x,y,t\right)=f(x,y)=\left\{\begin{array}{cl}( 1+\cos \left( 2\pi d\right), &d=\left | \left( x,y\right) -\left( \dfrac {1}{2},0\right) \right |  < \dfrac {1}{2} \\ 0 ,&otro  \end{array}\right.$$

In [19]:
function [u_save,x,t_save]=calor(k, M, t_f, tsteps) #resuelve la ecuacion del calor u_t = k u_xx en [0,1] con dato inicial x(1-x) y condiciones de contorno tipo Dirichlet u(0,t)=u(1,t)=0

#Datos de entrada:
# k: coeficiente k de la ecuacion
# M: numero de partes iguales en que se descompone [0,1]
# t_f: instante de tiempo hasta el que calculamos la solucion
# tsteps: numero de partes iguales en que se descompone [0,t_f]

#Mallado
dt=t_f/tsteps;
tiempos=0:dt:t_f;
h=1/M;
x=0:h:1;
    
#Datos iniciales
xred=x(2:M); #quitamos los extremos del vector x
u=(xred.*(1-xred)); #trasponemos para obtener un vector columna

#El operador D #Usamos el comando diag(vector,k) para crear una matriz tridiagonal
Tridiag=diag(-2*ones(M-1,1))+diag(ones(M-2,1),1)+diag(ones(M-2,1),-1);
D=(1/h^2)*Tridiag;

#Datos de salida: en vez de guardar todos los datos intermedios, guardamos el vector u solo Nframes veces.
Nframes=5;
marca=floor(tsteps/(Nframes-1));
u_save=zeros(M+1,Nframes);
t_save=zeros(1,Nframes); #Le ponemos las condiciones de frontera
u_save(1,:)=zeros(1,Nframes);
u_save(M+1,:)=zeros(1,Nframes); #guardamos la posicion de partida
u_save(2:M,1)=u;
t_save(1)=0;
    
#Bucle principal
I=eye(M-1);
A=(I+dt*k*D);
for n=1:tsteps
    u=A*u; #Guardamos los valores de u para algunos tiempos
    if mod(n,marca)==0
       indice=1+n/marca;
      u_save(2:M,indice)=u;
      t_save(indice)=tiempos(n);
    end
end

LoadError: syntax: expected "(" in function definition

Vamos a definir los parámetros para ejecutar el comando parabolic para nuestra ecuación;

In [17]:
function parabolic(u0,tlist,b,p,e,t,c,a,f,d) #u0 valor inicial, tlist es la lista de tiempos en los que guardamos la solución, tlist es la lista de tiempos en los que guardamos la solución,p,e,t que describen el mallado que usaremos para aproximar la solución y c,a,f,d  son coeficientes de la ecuación

    g=circleg; #Define el circulo D de centro 0 y radio 1
    b=circleb1; #Condiciones Dirichlet homogéneas
    c=.1;
    a=0;
    f=0;
    d=1; #Las variables p, e y t almacenan información sobre los vértices, los lados y los triángulos del mallado, respectivamente
    [p,e,t]=initmesh(g); #mallado
    t_f=1;
    Nframes=10; # Fija el número de intervalos de tiempo
    tlist=linspace(0,t_f,Nframes); # Fija la lista de tiempos: son Nframes 
#Definimos el dato inicial. p contiene los vértices de los triángulos del mallado siendo: p(1,:)las coordenadas x, y p(2,:) las coordenadas y de los vértices de los triángulos
    x=p(1,:);
    y=p(2,:);#Inicializamos u0 para que tome el valor 0
    u0=zeros(size(x)); #Calculamos el vector con las distancias al punto (0.5,0)
    distancias=sqrt((x-0.5).^2+(y).^2); #Buscamos los indices del vector de distancias donde la distancia es menor que 0.5
    I=find(distancias<0.5); #Para los indices anteriores, actualizamos el valor de u0 a 1-cos(d)
    fun=(1+cos(2*pi*distancias));
    u0(I)=fun(I); 
u=parabolic(u0,tlist,b,p,e,t,c,a,f,d);
end

LoadError: syntax: invalid assignment location "[p,e,t]"

Vamos a intentar resolver de forma exacta la misma ecuación con la condición de frontera y la distribución inicial de temperatura  ya mencionada antes.

En Matematicas Avanzadas para Física hemos visto que la solución general en un dominio circular la solución exacta de la
ecuación del calor en coordenadas polares viene dada por:
$$u\left( r,\theta ,t\right) =\sum ^{\infty }_{m=0}\sum ^{\infty }_{n=1}A_{mn}e^{\lambda _{mn}tk}\cos \left( m\theta \right) J_{m}\left( \sqrt {-\lambda _{mn}}r\right) +\sum ^{\infty }_{m=1}\sum ^{\infty }_{n=1}B_{mn}e^{\lambda _{mnk}t}\sin \left( m\theta \right) J_{m}\left( \sqrt {-\lambda _{mn}}r\right)$$

donde $\lambda _{mn}$ se puede escribir en la forma $\lambda _{mn}$ = − $\xi _{mn}$
, donde $\xi _{mn}$ es el cero n-ésimo de la función de Bessel J (la condición necesaria para que se verifique la condición de frontera $r = 1 \Rightarrow $  $u(r, \theta, t) = 0$ es $J_{m}\left( \sqrt {-\lambda _{mn}}\right) =J_{m}\left( \xi _{mn}\right) =0$. Los coeficientes $A _{mn}$  y $B _{mn}$  se
pueden calcular del siguiente modo:


$$A_{mn}=\dfrac {\int ^{2\pi }_{0}\int ^{1}_{0}u_{0}\left( r,\theta \right) \cos \left( m\theta \right) J_{m}\left( \xi _{mn}r\right) rdrd\theta }{\int ^{2\pi }_{0}\int ^{1}_{0}\left( \cos \left( m\theta \right) J_{mn}\left( \xi _{nm}r\right) \right) ^{2}rdrd\theta }$$

$$B_{mn}=\dfrac {\int ^{2\pi }_{0}\int ^{1}_{0}u_{0}\left( r,\theta \right) \sin \left( m\theta \right) J_{m}\left( \xi _{mn}r\right) rdrd\theta }{\int ^{2\pi }_{0}\int ^{1}_{0}\left( \sin \left( m\theta \right) J_{mn}\left( \xi _{nm}r\right) \right) ^{2}rdrd\theta }$$

Empezemos por definir la ec, y la malla del disco; para ello necesitamos los coeficientes:

In [3]:
function [A0,A,B]=coeficientes()
 #Devuelve los coeficientes de la funcion efe (definida en efe.m) en terminos de su expansion en terminos de senos, coseno y funciones de Bessel

#Primero introducimos a manos los ceros de las funciones de Bessel. Estos los obtuvimos de una tabla en e libro Diferential Parcial Equation de Asmar; 
besselC0n=[2.404825557695773,5.520078110286311,8.653727912911011,11.79153443901428,14.93091770848779,18.07106396791092,21.21163662987926,24.35247153074930,27.49347913204025,30.63460646843198];
besselCmn=[3.831705970207512,7.015586669815613,10.17346813506272,13.32369193631422,16.47063005087763,19.61585851046824,22.76008438059277,25.90367208761838,29.04682853491686,32.18967991097440;
5.135622301840683,8.417244140399855,11.61984117214906,14.79595178235126,17.95981949498783,21.11699705302185,24.27011231357310,27.42057354998456,30.56920449551640,33.71651950922270;
6.380161895923984,9.761023129981667,13.01520072169843,16.22346616031877,19.40941522643501,22.58272959310444,25.74816669929498,28.90835078092176,32.06485240709771,35.21867073861011;
7.588342434503804,11.06470948850119,14.37253667161759,17.61596604980483,20.82693295696239,24.01901952477111,27.19908776598125,30.37100766711725,33.53713771181922,36.69900112874465;
8.771483815959954,12.33860419746694,15.70017407971167,18.98013387517992,22.21779989656127,25.43034115422270,28.62661830729114,31.81171672404776,34.98878129455930,38.15986856196713;
9.936109524217684,13.58929017054122,17.00381966781601,20.32078921356651,23.58608443558139,26.82015198341140,30.03372238657047,33.23304176284712,36.42201966825846,39.60323941607540; 
11.08637001924509,14.82126872701317,18.28758283248173,21.64154101984840,24.93492788767302,28.19118845948320,31.42279419226558,34.63708935206932,37.83871738285361,41.03077369158554;
12.22509226400465,16.03777419088771,19.55453643099706,22.94517313187462,26.26681464117664,29.54565967099855,32.79580003734146,36.02561506386957,39.24044799517814,42.44388774327356;
13.35430047743524,17.24122038248913,20.80704778926411,24.23388525775055,27.58374896357301,30.88537896769667,34.15437792385510,37.40009997715659,40.62855371896453,43.84380142033735;
14.47550068655467,18.43346366696658,22.04698536469780,25.50945055418283,28.88737506353046,32.21185619971273,35.49990920537385,38.76180701788165,42.00419023667181,45.23157410353504];

#Numero de intervalos en que descomponemos los intervalos [0,1] y [0,2*pi]
M=50;
pasor=1/M;
r=0:pasor:1; #paso r 
pasotheta=2*pi/M; #paso theta
theta=0:pasotheta:2*pi;
[R,THETA]=meshgrid(r,theta);

#Primero calculamos los coeficientes A0
A0=zeros(10,1);
for n=1:10
    Cosenos=cos(0*THETA);
    Bessel=besselj(0,besselC0n(n)*R);
    integrando=efe(r,theta).*Cosenos.*Bessel.*R; #Integral por la regla del trapecio
    integral=0;
    for k=1:M
        for l=1:M
            integral = integral + pasor*pasotheta(integrando(k,l)+integrando(k+1,l)+...
                    integrando(k,l+1)+integrando(k+1,l+1))/2;
        end
    end
    #Ahora por la integral en el dividendo 
    #Valores de la funcion (cos(m*theta)*besselj(m,r))^2
    integrando=((Cosenos.*Bessel).^2).*R; #Integral por la regla del trapecio
    integral2=0;
    for k=1:M
        for l=1:M
            integral2 = integral2 + pasor*pasotheta*(integrando(k,l)+integrando(k+1,l)+...
                integrando(k,l+1)+integrando(k+1,l+1))/2;
        end
    end
    A0(n)=integral/integral2;
end
#Ahora calculamos los coeficientes A
A=zeros(10,10);
for m=1:10
    for n=1:10
        Cosenos=cos(m*THETA);
        Bessel=besselj(m,besselCmn(m,n)*R);
        integrando=efe(r,theta).*Cosenos.*Bessel.*R; #Integral por la regla del trapecio
        integral=0;
        for k=1:M
            for l=1:M
                integral = integral + pasor*pasotheta*(integrando(k,l)+integrando(k+1,l)+...
                    integrando(k,l+1)+integrando(k+1,l+1))/2;
            end
        end
        #Ahora por la integral en el dividendo
        #Valores de la funcion (cos(m*theta)*besselj(m,r))^2
        integrando=((Cosenos.*Bessel).^2).*R; #Integral por la regla del trapecio
        integral2=0;
        for k=1:M
            for l=1:M
                integral2 = integral2 + pasor*pasotheta*(integrando(k,l)+integrando(k+1,l)+...
                    integrando(k,l+1)+integrando(k+1,l+1))/2;
            end
        end
        A(m,n)=integral/integral2;
    end
end


#Finalmente, los coeficientes B
B=zeros(10,10);
for m=1:10
    for n=1:10
        Senos=sin(m*THETA);
        Bessel=besselj(m,besselCmn(m,n)*R);
        integrando=efe(r,theta).*Senos.*Bessel.*R; #Integral por la regla del trapecio
        integral=0;
        for k=1:M
            for l=1:M
                integral = integral + pasor*pasotheta*(integrando(k,l)+integrando(k+1,l)+...
                    integrando(k,l+1)+integrando(k+1,l+1))/2;
            end
        end
        #Ahora por la integral en el dividendo
        #Valores de la funcion (cos(m*theta)*besselj(m,r))^2
        integrando=((Senos.*Bessel).^2).*R; #Integral por la regla del trapecio
        integral2=0;
        for k=1:M
            for l=1:M
                integral2 = integral2 + pasor*pasotheta*(integrando(k,l)+integrando(k+1,l)+...
                    integrando(k,l+1)+integrando(k+1,l+1))/2;
            end
        end
        B(m,n)=integral/integral2;
    end
end

LoadError: syntax: expected "(" in function definition

In [ ]:
function Z=efe(r,theta) #Acepta un vector de N valores de r como primer argumento y un vector de M valores de theta como segundo argumento , devuelve una matriz NxM con los valores de el fichero "efe" en los puntos (r(n),theta(m))
[R,THETA]=meshgrid(r,theta);
X=R.*cos(THETA);
Y=R.*sin(THETA);
D=sqrt((X-0.5).^2+Y.^2);
I=find(D<0.5);
Fun=1+cos(2*pi*D);
Z=zeros(size(R));
Z(I)=Fun(I);
